# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [17]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [18]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

In [53]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

___
## Autenticando no  Twitter

* Conta: ***@datascience_terradonunca***

In [54]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [55]:
#Produto escolhido:
produto = 'pastel'

#Quantidade mínima de mensagens capturadas:
n = 550

#Quantidade mínima de mensagens para a base de treinamento:
t = 350

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [56]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [57]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [58]:
data = pd.read_excel("pastel.xlsx")
data2 = pd.read_excel("pastel1.xlsx")

In [59]:
s_repeticao = set(data.Treinamento)
s_repeticao2 = set(data2.Treinamento)

In [60]:
series = pd.Series(list(s_repeticao))
series2 = pd.Series(list(s_repeticao2))

In [61]:
a = pd.concat([series,series2])

In [62]:
a.to_excel('Treinamento.xlsx')

In [63]:
!open Treinamento.xlsx

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [87]:
data3 = pd.read_excel("pastel1.xlsx",sheet_name="Treinamento")

In [88]:
s = " ".join(data3[data3.Classe<=1].Treinamento)

In [89]:
pd.Series(s.lower().split()).value_counts(True)

pastel                     0.054259
de                         0.045179
e                          0.025724
a                          0.023779
eu                         0.023346
um                         0.021401
rt                         0.017942
o                          0.017077
comer                      0.012105
pra                        0.011673
com                        0.011241
da                         0.010592
que                        0.010592
mas                        0.010160
em                         0.009295
do                         0.009295
tô                         0.008214
gente                      0.007998
já                         0.007566
na                         0.007566
é                          0.007134
só                         0.006917
falando                    0.006701
vcs                        0.006485
também                     0.006485
sensação                   0.006269
relação                    0.006269
sozinhos?,                 0

In [79]:
data3.Treinamento.str.lower()

0      "tu quer comer pastel?"\nmeu pai chamando minh...
1      "vem comer pastel amor, a sobremesa é o seu do...
2      [5/9 00:57] juliaa: paga o pastel\n[5/9 00:58]...
3                       @_magrela__ vou comer pastel sim
4      @1640ginnie está com o que? estou bem!!!!! com...
5      @ahsuashaushauhs bo fazer uns pastel na minha ...
6                  @alinnefanelli e quem resiste pastel?
7      @alinnefanelli quero ver o pastel da feira com...
8      @alinnefanelli toda semana?! vocês não querem ...
9      @bomdia_sp a feira de salesopolis tem muito ma...
10       @caio_luis26 saudades de ir com vc comer pastel
11                @edupadelista pastel de nata? bom dia!
12                @ferigatto_gui vc não vai comer pastel
13                       @fp_nike aí já é frança, pastel
14     @g1sptv bom dia. feiras de rua tem que ter "pa...
15                  @geragerencia amiga tem pastel doce?
16     @gerardot3ixeira uma pena o fato de demorar um...
17     @guilhotinaaa vamo marca

In [77]:
data4 = cleanup(data3.lower())

AttributeError: 'DataFrame' object has no attribute 'lower'

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**